In [1]:
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np 
import os
import pandas as pd 
import shutil
import pathlib

In [2]:
# Create a directory if it doesn't exist
os.makedirs('./process/2nctoexcel', exist_ok=True)

# Define the path to the directory containing NetCDF files
path = "./process/1nc/"

for filename in os.listdir(path):
    file_location = os.path.join(path,filename)
    
    if os.path.isfile(file_location):
        print(file_location)
        
        # Open the NetCDF file using netCDF4
        f = netCDF4.Dataset(file_location)
        
        # Extract variables from the NetCDF file
        u10 = f.variables['u10']
        v10 = f.variables['v10']
        d2m = f.variables['d2m']
        t2m = f.variables['t2m']
        uvb = f.variables['uvb']
        e = f.variables['e']
        stl1 = f.variables['stl1']
        sp = f.variables['sp']
        tp = f.variables['tp']
        
        # Extract dimension information
        outputs = t2m.get_dims()
        k=0
        for i in outputs:

            print(i.name)
            if(i.name=='time'):
                time_dim = outputs[k]
            elif(i.name=='latitude'):
                lat_dim = outputs[k]
            elif(i.name=='longitude'):
                lon_dim = outputs[k]
            k=k+1
            
        # Print dimension information
        print('dimension information')
        print('time  ',time_dim)
        print('lat  ',lat_dim)
        print('long  ',lon_dim)
        
        # Extract data from dimensions and variables
        time_var = f.variables[time_dim.name]
        times = num2date(time_var[:], time_var.units)
        latitudes = f.variables[lat_dim.name][:]
        longitudes = f.variables[lon_dim.name][:]
        
        # Define the output directory and filename
        output_dir = './'
        filename=filename.split(".")[0]
        filename = os.path.join(output_dir, r'./process/2nctoexcel/'+filename+'.xlsx')
        print(f'Writing data in tabular form to {filename} (this may take some time)...')
        
        # Flatten the data and remove None values
        times_grid, latitudes_grid, longitudes_grid = [
            x.flatten() for x in np.meshgrid(times, latitudes, longitudes, indexing='ij')]
        
        u10 = list(filter(None, u10[:].flatten()))
        v10 = list(filter(None, v10[:].flatten()))
        d2m = list(filter(None, d2m[:].flatten()))
        t2m = list(filter(None, t2m[:].flatten()))
        # uvb = list(filter(None, uvb[:].flatten()))
        e = list(filter(None, e[:].flatten()))
        stl1 = list(filter(None, stl1[:].flatten()))
        sp = list(filter(None, sp[:].flatten()))
        # tp = list(filter(None, tp[:].flatten()))

        tp2 = []
        for ele in tp[:].flatten():
            if ele != None:
                tp2.append(ele)

        uvb2 = []
        for ele in uvb[:].flatten():
            if ele != None:
                uvb2.append(ele)


                
                
        # Create a DataFrame with the extracted data
        df = pd.DataFrame({
            'time': [t.isoformat() for t in times_grid],
            'latitude': latitudes_grid,
            'longitude': longitudes_grid,
            'u10': u10,
            'v10': v10,
            'd2m': d2m,
            't2m': t2m,
            'uvb': uvb2,
            'e': e,
            'stl1': stl1,
            'sp': sp,
            'tp': tp2,
        })
        
        # Sort the DataFrame by the 'time' column
        df = df.sort_values(by='time')

        # Write the data to an Excel file
        df.to_excel(filename, engine='xlsxwriter', index=False)
        print('Done')
        print('------------')

./process/1nc/1-2022-11-25-คองแห-หาดใหญ่-สงขลา.nc


FileNotFoundError: [Errno 2] No such file or directory: './process/1nc/1-2022-11-25-คองแห-หาดใหญ่-สงขลา.nc'